In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd
from collections import defaultdict
import glob
from PIL import Image
from IPython.display import display, HTML

scans_dir = []

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    if dirname.endswith('scans'):
        scans_dir.append(dirname)
        
print(f'Found {len(scans_dir)} scans directories')

In [ ]:
df = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/train.csv')

df.head()

In [ ]:
def rle_decode(rles, height, width, fill=255): # https://www.kaggle.com/code/mohammaddehghan/pytorch-dataloader-and-visualize
    mask_t = np.zeros((width, height, 3), dtype=np.uint8)
    
    for ii, rle in enumerate(rles):
        if not isinstance(rle, str):
            continue

        s = rle.split()
        start, length = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
        start -= 1
        mask = np.zeros(height*width, dtype=np.uint8)
        for i, l in zip(start, length):
            mask[i:i+l] = fill
        mask = mask.reshape(width,height)#.T
        mask = np.ascontiguousarray(mask)

        mask_t[:, :, ii] = mask
    
    return mask_t

def openImage(path):
    img = np.array(Image.open(path))
    return Image.fromarray(((img / img.max()) * 255).astype(np.uint8))

def getdfIdx(idx):
    return df[df.apply(lambda x: scans_dir[idx].split('/')[-2] in x.id, axis=1)].groupby(['id']).segmentation.apply(list).reset_index()

In [ ]:
f1 = [fn for fn in scans_dir if 'case7_day0' in fn][0]

scans_dir.index(f1)

In [ ]:
SCAN_IDX = 5

fns = sorted(glob.glob(f'{scans_dir[SCAN_IDX]}/*'))

imgs = (openImage(f) for f in fns)
img = next(imgs)
fn = scans_dir[SCAN_IDX].split('/')[-2] + '.gif'

h, w = fns[0].split('/')[-1].split('_')[2:-2]
h, w = int(h), int(w)

masks = (Image.fromarray(rle_decode(m, h, w)) for m in getdfIdx(SCAN_IDX).segmentation.values)
mask = next(masks)

img.save(fp=fn, format='GIF', append_images=imgs, save_all=True, duration=200, loop=0)

mask_fn = 'mask_' + fn

mask.save(fp=mask_fn, format='GIF', append_images=masks, save_all=True, duration=200, loop=0)

In [ ]:
fn, mask_fn

In [ ]:
HTML(f'<img src="{fn}"> <img src="{mask_fn}">')